In [ ]:
import tensorflow.compat.v1 as tf
tf.disable_v2_behavior()
tf.set_random_seed(1)
config = tf.compat.v1.ConfigProto()
config.gpu_options.allow_growth=True
sess = tf.compat.v1.Session(config=config)
import matplotlib.pyplot as plt
import PIL.Image as Image
import math
import numpy as np
import tensorflow.keras.backend as K
from tensorflow.keras.utils import plot_model
import json
from tensorflow.keras.losses import mse
import os
import glob
import random
#import cv2
from sklearn.model_selection import train_test_split
from tensorflow.keras.applications.inception_v3 import InceptionV3
import datetime
from tensorflow.keras.layers import Dense, Input, concatenate, Conv2D, MaxPooling2D, Flatten, Lambda
from tensorflow.keras.models import Model
from tensorflow.keras import optimizers
from tensorflow.keras.callbacks import ModelCheckpoint, EarlyStopping, CSVLogger, ReduceLROnPlateau
from tensorflow.keras.callbacks import TensorBoard
%matplotlib inline

In [3]:
def add_layer(tensor):
    return tensor[0] + tensor[1]

def mul_layer(tensor):
    return tensor[0] * tensor[1]

def div_layer(tensor):
    return tensor[0] / tensor[1]

def sub_layer(tensor):
    return tensor[0] - tensor[1]

def neg_layer(tensor):
    return -tensor

def cos_layer(tensor):
    return tf.math.cos(tensor)

def sin_layer(tensor):
    return tf.math.sin(tensor)

In [ ]:
# feature extraction from left image

left_img = Input(shape = (150,150,3), name="left_image")

# feature extraction from right image
right_img = Input(shape = (150,150,3), name="right_image")

left_phi_model = InceptionV3(weights='imagenet', include_top=False, input_tensor=left_img, input_shape=(150,150,3))
for layer in left_phi_model.layers:
  layer._name = layer._name+"f1_base"

left_phi_features = left_phi_model.output
left_flat = Flatten(name='left-phi-flattened')(left_phi_features)

right_phi_model = InceptionV3(weights='imagenet', include_top=False, input_tensor=right_img, input_shape=(150,150,3))
for layer in right_phi_model.layers:
  layer._name = layer._name+"f2_base"

right_phi_features = right_phi_model.output
right_flat = Flatten(name='right-phi-flattened')(right_phi_features)

concat = concatenate([left_flat, right_flat])


# alpha_xworld
dense_31 = Dense(120, activation = 'relu')(concat)
dense_32 = Dense(84, activation = 'relu')(dense_31)
alpha_xworld = Dense(1, name='w_xworld', activation = 'sigmoid')(dense_32)

# alpha_yworld
dense_33 = Dense(120, activation = 'relu')(concat)
dense_34 = Dense(84, activation = 'relu')(dense_33)
alpha_yworld = Dense(1, name='w_yworld', activation = 'sigmoid')(dense_34)

# alpha_zworld
dense_35 = Dense(120, activation = 'relu')(concat)
dense_36 = Dense(84, activation = 'relu')(dense_35)
alpha_zworld = Dense(1, name='w_zworld', activation = 'sigmoid')(dense_36)

# alpha_fx
dense_alpha_fx = Dense(120, activation = 'relu')(concat)
dense_alpha_fx2 = Dense(84, activation = 'relu')(dense_alpha_fx)
alpha_fx = Dense(1, name='alpha_fx', activation = 'sigmoid')(dense_alpha_fx2)

# alpha_fy
dense_alpha_fy = Dense(120, activation = 'relu')(concat)
dense_alpha_fy2 = Dense(84, activation = 'relu')(dense_alpha_fy)
alpha_fy = Dense(1, name='alpha_fy', activation = 'sigmoid')(dense_alpha_fy2)

# alpha_u0
dense_alpha_u0 = Dense(120, activation = 'relu')(concat)
dense_alpha_u02 = Dense(84, activation = 'relu')(dense_alpha_u0)
alpha_u0 = Dense(1, name='alpha_u0', activation = 'sigmoid')(dense_alpha_u02)

# alpha_v0
dense_alpha_v0 = Dense(120, activation = 'relu')(concat)
dense_alpha_v02 = Dense(84, activation = 'relu')(dense_alpha_v0)
alpha_v0 = Dense(1, name='alpha_v0', activation = 'sigmoid')(dense_alpha_v02)

# alpha_tx
dense_alpha_tx = Dense(120, activation = 'relu')(concat)
dense_alpha_tx2 = Dense(84, activation = 'relu')(dense_alpha_tx)
alpha_tx = Dense(1, name='alpha_tx', activation = 'sigmoid')(dense_alpha_tx2)

# alpha_ty
dense_alpha_ty = Dense(120, activation = 'relu')(concat)
dense_alpha_ty2 = Dense(84, activation = 'relu')(dense_alpha_ty)
alpha_ty = Dense(1, name='alpha_ty', activation = 'sigmoid')(dense_alpha_ty2)

# alpha_tz
dense_alpha_tz = Dense(120, activation = 'relu')(concat)
dense_alpha_tz2 = Dense(84, activation = 'relu')(dense_alpha_tz)
alpha_tz = Dense(1, name='alpha_tz', activation = 'sigmoid')(dense_alpha_tz2)

# alpha_baseline
dense_alpha_baseline = Dense(120, activation = 'relu')(concat)
dense_alpha_baseline2 = Dense(84, activation = 'relu')(dense_alpha_baseline)
alpha_baseline = Dense(1, name='alpha_baseline', activation = 'sigmoid')(dense_alpha_baseline2)

# alpha_disparity
dense_alpha_disparity = Dense(120, activation = 'relu')(concat)
dense_alpha_disparity2 = Dense(84, activation = 'relu')(dense_alpha_disparity)
alpha_disparity = Dense(1, name='alpha_disparity', activation = 'sigmoid')(dense_alpha_disparity2)

# alpha_pitch
dense_alpha_pitch = Dense(120, activation = 'relu')(concat)
dense_alpha_pitch2 = Dense(84, activation = 'relu')(dense_alpha_pitch)
alpha_pitch = Dense(1, name='alpha_pitch', activation = 'sigmoid')(dense_alpha_pitch2)


# fx
dense_1 = Dense(120, activation = 'relu')(concat)
dense_2 = Dense(84, activation = 'relu')(dense_1)
pred_fx3 = Dense(1, name='fx3')(dense_2)
pred_fx = pred_fx3 

# fy
dense_3 = Dense(120, activation = 'relu')(concat)
dense_4 = Dense(84, activation = 'relu')(dense_3)
pred_fy3 = Dense(1, name='fy3')(dense_4)
pred_fy = pred_fy3 

# u0
dense_5 = Dense(120, activation = 'relu')(concat)
dense_6 = Dense(84, activation = 'relu')(dense_5)
pred_u03 = Dense(1, name='u03')(dense_6)
pred_u0 = Lambda(mul_layer, name='u0')([alpha_u0, pred_u03])


# v0
dense_7 = Dense(120, activation = 'relu')(concat)
dense_8 = Dense(84, activation = 'relu')(dense_7)
pred_v03 = Dense(1, name='v03')(dense_8)
pred_v0 = Lambda(mul_layer, name='v0')([alpha_v0, pred_v03])


# baseline
dense_9 = Dense(120, activation = 'relu')(concat)
dense_10 = Dense(84, activation = 'relu')(dense_9)
pred_baseline3 = Dense(1, name='baseline3')(dense_10)
pred_baseline = Lambda(mul_layer, name='baseline')([alpha_baseline, pred_baseline3])


# tx
dense_11 = Dense(120, activation = 'relu')(concat)
dense_12 = Dense(84, activation = 'relu')(dense_11)
pred_x3 = Dense(1, name='x3')(dense_12)
pred_x = Lambda(mul_layer, name='x')([alpha_tx, pred_x3])

# ty
dense_13 = Dense(120, activation = 'relu')(concat)
dense_14 = Dense(84, activation = 'relu')(dense_13)
pred_y3 = Dense(1, name='y3')(dense_14)
pred_y = Lambda(mul_layer, name='y')([alpha_ty, pred_y3])

# tz
dense_15 = Dense(120, activation = 'relu')(concat)
dense_16 = Dense(84, activation = 'relu')(dense_15)
pred_z3 = Dense(1, name='z3')(dense_16)
pred_z = Lambda(mul_layer, name='z')([alpha_tz, pred_z3])


# pitch
dense_17 = Dense(120, activation = 'relu')(concat)
dense_18 = Dense(84, activation = 'relu')(dense_17)
pred_pitch3 = Dense(1, name='pitch3')(dense_18)
pred_pitch = Lambda(mul_layer, name='pitch')([alpha_pitch, pred_pitch3])

# u
dense_19 = Dense(120, activation = 'relu')(concat)
dense_20 = Dense(84, activation = 'relu')(dense_19)
pred_u = Dense(1, name='u')(dense_20)

# v
dense_21 = Dense(120, activation = 'relu')(concat)
dense_22 = Dense(84, activation = 'relu')(dense_21)
pred_v = Dense(1, name='v')(dense_22)

# disparity
dense_23 = Dense(120, activation = 'relu')(concat)
dense_24 = Dense(84, activation = 'relu')(dense_23)
pred_disparity3 = Dense(1, name='disparity3')(dense_24)
pred_disparity = Lambda(mul_layer, name='disparity')([alpha_disparity, pred_disparity3])




# xCam = (self.intrinsic.fx * self.extrinsic.baseline) / disparity
mul_1 = Lambda(mul_layer)([pred_fx, pred_baseline])
xCam = Lambda(div_layer)([mul_1, pred_disparity])
#xCam = Lambda(mul_layer, name='xCam')([xCam, w_xcam])

# yCam = - (xCam / self.intrinsic.fx) * (u - self.intrinsic.u0)
div_1 = Lambda(div_layer)([xCam, pred_fx])
sub_1 = Lambda(sub_layer)([pred_u, pred_u0])
yCam = Lambda(mul_layer)([Lambda(neg_layer)(div_1), sub_1])
#yCam = Lambda(mul_layer, name='yCam')([yCam, w_ycam])

# zCam = (xCam / self.intrinsic.fy) * (self.intrinsic.v0 - v)
div_2 = Lambda(div_layer)([xCam, pred_fy])
sub_2 = Lambda(sub_layer)([pred_v0, pred_v])
zCam = Lambda(mul_layer)([div_2, sub_2])
#zCam = Lambda(mul_layer, name='zCam')([zCam, w_zcam])

# Y = yCam + self.extrinsic.y
pred_yWorld2 = Lambda(add_layer)([yCam, pred_y])
pred_yWorld = Lambda(mul_layer, name='yWorld')([pred_yWorld2, alpha_yworld])

# X = xCam * math.cos(self.extrinsic.pitch) + zCam * math.sin(self.extrinsic.pitch) + self.extrinsic.x
mul_2 = Lambda(mul_layer)([xCam, Lambda(cos_layer)(pred_pitch)])
mul_3 = Lambda(mul_layer)([zCam, Lambda(sin_layer)(pred_pitch)])
add_1 = Lambda(add_layer)([mul_2, mul_3])
pred_xWorld2 = Lambda(add_layer)([add_1, pred_x])
pred_xWorld = Lambda(mul_layer, name='xWorld')([pred_xWorld2, alpha_xworld])

# Z = - xCam * math.sin(self.extrinsic.pitch) + zCam * math.cos(self.extrinsic.pitch) + self.extrinsic.z
mul_4 = Lambda(mul_layer)([Lambda(neg_layer)(xCam), Lambda(sin_layer)(pred_pitch)])
mul_5 = Lambda(mul_layer)([zCam, Lambda(cos_layer)(pred_pitch)])
add_2 = Lambda(add_layer)([mul_4, mul_5])
pred_zWorld2 = Lambda(add_layer)([add_2, pred_z])
pred_zWorld = Lambda(mul_layer, name='zWorld')([pred_zWorld2, alpha_zworld])

# create model
model = Model(inputs=[left_img, right_img], outputs=[pred_fx, pred_fy, pred_u0, pred_v0, pred_baseline, pred_disparity, pred_x, pred_y, pred_z, pred_pitch, pred_xWorld,pred_yWorld,pred_zWorld])

# set output types
target1 = tf.placeholder(dtype='float32', shape=(1,1))
target2 = tf.placeholder(dtype='float32', shape=(1,1))
target3 = tf.placeholder(dtype='float32', shape=(1,1))
target4 = tf.placeholder(dtype='float32', shape=(1,1))
target5 = tf.placeholder(dtype='float32', shape=(1,1))
target6 = tf.placeholder(dtype='float32', shape=(1,1))
target7 = tf.placeholder(dtype='float32', shape=(1,1))
target8 = tf.placeholder(dtype='float32', shape=(1,1))
target9 = tf.placeholder(dtype='float32', shape=(1,1))
target10 = tf.placeholder(dtype='float32', shape=(1,1))
target11 = tf.placeholder(dtype='float32', shape=(1,1))
target12 = tf.placeholder(dtype='float32', shape=(1,1))
target13 = tf.placeholder(dtype='float32', shape=(1,1))

# get model summary
#model.summary()

learning_rate = 0.001

# compile model
model.compile(loss="mae", target_tensors=[target1, target2, target3, target4, target5, target6, target7, target8, target9, target10, target11, target12, target13],optimizer=optimizers.legacy.Adam(lr=learning_rate));
#plot_model(model, to_file='model.png')

In [5]:
import numpy as np
data_path = 'your pth/'
Fx = np.load(data_path+"fx.npy", mmap_mode='r')
Fy = np.load(data_path+"fy.npy", mmap_mode='r')
U0 = np.load(data_path+"U0.npy", mmap_mode='r')
V0 = np.load(data_path+"V0.npy", mmap_mode='r')
Disparity = np.load(data_path+"disparity.npy", mmap_mode='r')
Tx = np.load(data_path+"x.npy", mmap_mode='r')
Ty = np.load(data_path+"y.npy", mmap_mode='r') 
Tz = np.load(data_path+"z.npy", mmap_mode='r')
Baseline = np.load(data_path+"baseline.npy", mmap_mode='r')
Pitch = np.load(data_path+"pitch.npy", mmap_mode='r')
X = np.load(data_path+"xworld.npy", mmap_mode='r')
Y = np.load(data_path+"yworld.npy", mmap_mode='r')
Z = np.load(data_path+"zworld.npy", mmap_mode='r')
Left_images = np.load(data_path+"left_img.npy", mmap_mode='r')
Right_images = np.load(data_path+"right_img.npy", mmap_mode='r')

In [8]:
import os
from tensorflow.keras.callbacks import TensorBoard, LearningRateScheduler
from keras.applications.inception_v3 import InceptionV3
from keras.applications.imagenet_utils import preprocess_input
from keras.models import Model
from keras.layers import Dense, Flatten, Input
#from utils_regressor_focal_dist import RotNetDataGenerator, angle_error, CustomModelCheckpoint
from keras import optimizers
import numpy as np
import glob, math
from shutil import copyfile
import datetime, random
import tensorflow as tf
#from keras.backend.tensorflow_backend import set_session

model_name = 'model_multi_class/'
SAVE = "CPL_A/" + datetime.datetime.now().strftime("%Y%m%d-%H%M%S") + '/'
# Save
output_folder = SAVE + model_name
if not os.path.exists(output_folder):
    os.makedirs(output_folder)

output_log = output_folder + "Log/"
if not os.path.exists(output_log):
    os.makedirs(output_log)

output_weight = output_folder + "Best/"
if not os.path.exists(output_weight):
    os.makedirs(output_weight)


input_shape = (112, 112, 3)

tensorboard = TensorBoard(log_dir=output_log)

early_stopping = EarlyStopping(monitor='val_loss', patience=5, verbose=0, mode='auto', restore_best_weights=True)

csv_logger = CSVLogger('CPL_a_training_100.log')


checkpointer = tf.keras.callbacks.ModelCheckpoint(
    filepath=output_weight + "weights_{epoch:02d}_{val_loss:.2f}.h5",
    monitor="val_loss",
    verbose=0,
    save_best_only=True,
    save_weights_only=True
)

history = model.fit(
    x=[Left_images[:int(len(Left_images)*0.7)], Right_images[:int(len(Left_images)*0.7)]],
    y=[Fx[:int(len(Left_images)*0.7)], Fy[:int(len(Left_images)*0.7)], U0[:int(len(Left_images)*0.7)], V0[:int(len(Left_images)*0.7)], Baseline[:int(len(Left_images)*0.7)], Disparity[:int(len(Left_images)*0.7)], Tx[:int(len(Left_images)*0.7)], Ty[:int(len(Left_images)*0.7)], Tz[:int(len(Left_images)*0.7)], Pitch[:int(len(Left_images)*0.7)], X[:int(len(Left_images)*0.7)], Y[:int(len(Left_images)*0.7)], Z[:int(len(Left_images)*0.7)]],
    batch_size=32,
    epochs=100,
    verbose=2,
    shuffle=True,
    initial_epoch=0,
    validation_data=([Left_images[int(len(Left_images)*0.7):], Right_images[int(len(Left_images)*0.7):]], [Fx[int(len(Left_images)*0.7):], Fy[int(len(Left_images)*0.7):], U0[int(len(Left_images)*0.7):], V0[int(len(Left_images)*0.7):], Baseline[int(len(Left_images)*0.7):], Disparity[int(len(Left_images)*0.7):], Tx[int(len(Left_images)*0.7):], Ty[int(len(Left_images)*0.7):], Tz[int(len(Left_images)*0.7):], Pitch[int(len(Left_images)*0.7):], X[int(len(Left_images)*0.7):], Y[int(len(Left_images)*0.7):], Z[int(len(Left_images)*0.7):]]),
    callbacks=[tensorboard, checkpointer, csv_logger]
)

Train on 44520 samples, validate on 19080 samples
Epoch 1/100


/opt/conda/lib/python3.10/site-packages/keras/src/engine/training_v1.py:2335: UserWarning: `Model.state_updates` will be removed in a future version. This property should not be used in TensorFlow 2.0, as `updates` are applied automatically.
  updates = self.state_updates


44520/44520 - 656s - loss: 211.5774 - fx3_loss: 13.3710 - fy3_loss: 13.3744 - u0_loss: 75.0004 - v0_loss: 2.8196 - baseline_loss: 0.6996 - disparity_loss: 4.5347 - x_loss: 0.4034 - y_loss: 0.4491 - z_loss: 0.2502 - pitch_loss: 0.0870 - xWorld_loss: 39.3608 - yWorld_loss: 30.9094 - zWorld_loss: 30.5330 - val_loss: 1361.7572 - val_fx3_loss: 469.8192 - val_fy3_loss: 365.6014 - val_u0_loss: 75.0000 - val_v0_loss: 270.4667 - val_baseline_loss: 0.7009 - val_disparity_loss: 66.8987 - val_x_loss: 0.4020 - val_y_loss: 0.4506 - val_z_loss: 0.2496 - val_pitch_loss: 0.0877 - val_xWorld_loss: 44.1244 - val_yWorld_loss: 34.1249 - val_zWorld_loss: 33.8074 - 656s/epoch - 15ms/sample
Epoch 2/100
44520/44520 - 318s - loss: 201.6686 - fx3_loss: 9.3577 - fy3_loss: 9.3028 - u0_loss: 75.0000 - v0_loss: 1.2582 - baseline_loss: 0.6998 - disparity_loss: 4.2770 - x_loss: 0.4017 - y_loss: 0.4490 - z_loss: 0.2502 - pitch_loss: 0.0869 - xWorld_loss: 39.2567 - yWorld_loss: 30.8381 - zWorld_loss: 30.4621 - val_loss: